In [1]:
print "foo"

foo


In [2]:
import pandas as pd
import os
import pysam
import numpy as np
import pybedtools


import subprocess

from gscripts.encode import encode_helpers
from gscripts import qtools

In [3]:
merged_data = encode_helpers.get_merged_data()

#For Peak Analysis we only want to analyze datasets that have been submitted
merged_data = merged_data[merged_data.submitted]
merged_data = merged_data[['CLIP', 'INPUT', 'input_norm']]

In [4]:
outdir = "/projects/ps-yeolab3/encode/analysis/normalized_wigs_v2"

def normalize_wig(ip_fn, input_fn):
    ip_base = os.path.splitext(os.path.basename(ip_fn))[0]
    input_base = os.path.splitext(os.path.basename(input_fn))[0]
    print input_base 
    
    ip_scaling_factor = 1000000. / pysam.Samfile(ip_fn).mapped
    input_scapling_factor = 1000000. / pysam.Samfile(input_fn).mapped
        
    ip_bg = "{}/{}.bg".format(outdir, ip_base)
    input_bg = "{}/{}.bg".format(outdir, input_base)
    
    ip_offset_wig = "{}/{}.offset.wig".format(outdir, ip_base)
    input_offset_wig = "{}/{}.offset.wig".format(outdir, input_base)
    
    ip_offset_bw = "{}/{}.offset.bw".format(outdir, ip_base)
    input_offset_bw = "{}/{}.offset.bw".format(outdir, input_base)
    
    normalized_wig = "{}/{}.normalized.wig".format(outdir, ip_base)
    normalized_bw = "{}/{}.normalized.bw".format(outdir, ip_base)

    normalized_bg = "{}/{}.normalized.bg".format(outdir, ip_base)
    
    !bedtools genomecov -ibam $ip_fn -bg -split -scale $ip_scaling_factor -g /projects/ps-yeolab/genomes/hg19/hg19.chrom.sizes > $ip_bg
    !bedtools genomecov -ibam $input_fn -bg -scale $input_scapling_factor -split -g /projects/ps-yeolab/genomes/hg19/hg19.chrom.sizes > $input_bg
    !/home/gpratt/software/WiggleTools/bin/wiggletools offset 1  $ip_bg > $ip_offset_wig
    !/home/gpratt/software/WiggleTools/bin/wiggletools offset 1 $input_bg > $input_offset_wig

    !wigToBigWig $ip_offset_wig /projects/ps-yeolab/genomes/hg19/hg19.chrom.sizes $ip_offset_bw
    !wigToBigWig $input_offset_wig /projects/ps-yeolab/genomes/hg19/hg19.chrom.sizes $input_offset_bw

    !/home/gpratt/software/WiggleTools/bin/wiggletools ratio $ip_offset_bw default 1 $input_offset_bw > $normalized_wig

    !wigToBigWig $normalized_wig /projects/ps-yeolab/genomes/hg19/hg19.chrom.sizes $normalized_bw
    !bigWigToBedGraph $normalized_bw $normalized_bg

In [5]:
clips = master_processing_table.dropna(subset=["CLIP_rep1"], axis=0)

In [ ]:
for name, row in clips.iterrows():
    ip_base = os.path.splitext(os.path.basename(row.CLIP_rep1))[0]
    normalized_bw = "{}/{}.normalized.bw".format(outdir, ip_base)
    if os.path.exists(normalized_bw):
        print ip_base, "exists"
        continue

    try:
        normalize_wig(row.CLIP_rep1, row.INPUT)
        normalize_wig(row.CLIP_rep2, row.INPUT)
    except Exception as e:
        print e
        pass

PTBP1-202-INPUT_S6_R1.unassigned.adapterTrim.round2.rmRep.rmDup.sorted.r2


# Other approach

In [4]:
bam = merged_data.CLIP[0]
input_bam = merged_data.INPUT[0]

In [5]:
import tempfile

In [27]:
bam = "/home/gpratt/ad-hoc/chrM_test.bam"
input_bam = "/home/gpratt/ad-hoc/chrM_input_test.bam"

In [6]:
#Generated both dataframes
bedtool = pybedtools.BedTool(bam)
input_bedtool = pybedtools.BedTool(input_bam)

df = bedtool.genome_coverage(dz=True, split=True, strand="+").to_dataframe(names=['chrom', 'loc','coverage'])
# neg_df = bedtool.genome_coverage(dz=True, split=True, strand="-").to_dataframe(names=['chrom', 'loc',  'coverage'])

input_df = input_bedtool.genome_coverage(dz=True, split=True, strand="+").to_dataframe(names=['chrom', 'loc',  'coverage'])
# input_neg_df = input_bedtool.genome_coverage(dz=True, split=True, strand="-").to_dataframe(names=['chrom', 'loc', 'coverage'])


In [7]:
#Merged Dataframes
merged_df = pd.merge(df, input_df, how='outer', 
                     left_on=['chrom', 'loc', ],
                     right_on=['chrom', 'loc', ],
                     suffixes=['_ip', '_input'],
                    )

# merged_neg_df = pd.merge(neg_df, input_neg_df, how='outer', 
#                          left_on=['chrom', 'loc',],
#                          right_on=['chrom', 'loc',],
#                          suffixes=['_ip', '_input'],
#                         )

#Add a pesudocount
merged_df = merged_df.fillna(0)
merged_df['coverage_ip'] += 1
merged_df['coverage_input'] += 1

# merged_neg_df = merged_neg_df.fillna(0)
# merged_neg_df['coverage_ip'] += 1
# merged_neg_df['coverage_input'] += 1

#Compute total coverage across all regions
total_ip = merged_df.coverage_ip.sum() #+ merged_neg_df.coverage_ip.sum()
total_input = merged_df.coverage_input.sum() #+ merged_neg_df.coverage_input.sum()

scale = 1000000

#calculate probablity of IP and input (we scale it to avoid numerical percision issues)
merged_df['p_ip'] = (merged_df.coverage_ip / total_ip) * scale
merged_df['p_input'] = (merged_df.coverage_input / total_input) * scale

# merged_neg_df['p_ip'] = (merged_neg_df.coverage_ip / total_ip) * scale
# merged_neg_df['p_input'] = (merged_neg_df.coverage_input / total_input) * scale

In [8]:
merged_df['scaled_entropy'] = (merged_df['p_ip']) * (np.log2(merged_df['p_ip'] ) - np.log2(merged_df['p_input']))
# merged_neg_df['scaled_entropy'] = (merged_neg_df['p_ip']) * (np.log2(merged_neg_df['p_ip']) - np.log2(merged_neg_df['p_input']))

In [9]:
# merged_neg_df.loc[merged_neg_df['scaled_entropy'] < 0, 'scaled_entropy'] = 0

In [13]:
merged_df['start'] = (merged_df['loc'] - 1).astype(int)
merged_df['stop'] = merged_df['loc'].astype(int)

# merged_neg_df['start'] = merged_neg_df['loc'] - 1
# merged_neg_df['stop'] = merged_neg_df['loc']

In [14]:
handle_pos, fn_pos = tempfile.mkstemp()
handle_neg, fn_neg = tempfile.mkstemp()

In [19]:
foo = merged_df.head()

In [20]:
foo.sort_values(['chrom', 'start', 'stop'])

,chrom,loc,coverage_ip,coverage_input,p_ip,p_input,scaled_entropy,start,stop
0,chr1,94842.0,2.0,1.0,0.004633,0.003584,0.001717,94841,94842
1,chr1,94843.0,2.0,1.0,0.004633,0.003584,0.001717,94842,94843
2,chr1,94844.0,2.0,1.0,0.004633,0.003584,0.001717,94843,94844
3,chr1,94845.0,2.0,1.0,0.004633,0.003584,0.001717,94844,94845
4,chr1,94846.0,2.0,1.0,0.004633,0.003584,0.001717,94845,94846


In [22]:
foo = merged_df.sort_values(['chrom', 'start', 'stop'])

In [25]:
grp = foo.groupby("chrom")

In [39]:
for name, df in grp:
    print name, all(df.start[:-1] < df.start[1:].values)

chr1 True
chr10 True
chr11 True
chr12 True
chr13 True
chr14 True
chr15 True
chr16 True
chr17 True
chr18 True
chr19 True
chr2 True
chr20 True
chr21 True
chr22 True
chr3 True
chr4 True
chr5 True
chr6 True
chr7 True
chr8 True
chr9 True
chrM True
chrX True
chrY True


In [29]:
chrY_grp = grp.get_group("chrY")

In [ ]:

[gpratt@tscc-1-32 scripts]$ bedGraphToBigWig ~/ad-hoc/foo.bg /projects/ps-yeolab/genomes/hg19/hg19.chrom.sizes /projects/ps-yeolab3/encode/analysis/entropy_bw/205_01_IGF2BP1.merged.r2.entropy.pos.bw
Start (4294967295) after end (0) line 49382842 of /home/gpratt/ad-hoc/foo.bg


In [33]:
bar = foo[foo.start == 4294967295]

In [34]:
bar

,chrom,loc,coverage_ip,coverage_input,p_ip,p_input,scaled_entropy,start,stop


True

In [ ]:
foo[['chrom', 'start', 'stop', 'scaled_entropy']].to_csv(fn_pos, sep="\t", header=False, index=False, chunksize=40000)


In [13]:
merged_df.sort_index()

In [114]:
len(merged_df)

151588138

In [ ]:
151588138
151588138

In [ ]:
merged_df

In [ ]:
merged_df[['chrom', 'start', 'stop', 'scaled_entropy']].to_csv(fn_pos, sep="\t", header=False, index=False, chunksize=20000)
merged_neg_df[['chrom', 'start', 'stop', 'scaled_entropy']].to_csv(fn_neg, sep="\t", header=False, index=False, chunksize=20000)

In [ ]:
print "foo"

In [38]:
bw_pos = "foo.pos.bw"
bw_neg = 'foo.neg.bw'

In [45]:
cmd = "bedGraphToBigWig {} /projects/ps-yeolab/genomes/hg19/hg19.chrom.sizes {}".format(fn_pos, bw_pos)
p1 = subprocess.Popen(cmd, shell=True)
print cmd
p1.wait()

cmd = "bedGraphToBigWig {} /projects/ps-yeolab/genomes/hg19/hg19.chrom.sizes {}".format(fn_neg, bw_neg)
p1 = subprocess.Popen(cmd, shell=True)
print cmd
p1.wait()

wigToBigWig -keepAllChromosomes /state/partition1/gpratt/8681503.tscc-mgr.local/tmpTuEV9f /projects/ps-yeolab/genomes/hg19/hg19.chrom.sizes foo.pos.bw
wigToBigWig -keepAllChromosomes /state/partition1/gpratt/8681503.tscc-mgr.local/tmpWu8Hv2 /projects/ps-yeolab/genomes/hg19/hg19.chrom.sizes foo.neg.bw


0

In [90]:
out_dir = "/projects/ps-yeolab3/encode/analysis/entropy_bw"
def run_entropy_bw(clip_fn, input_fn):
    raw_name = os.path.splitext(os.path.basename(clip_fn))[0]
    pos_name = os.path.join(out_dir, raw_name + ".entropy.pos.bw")
    neg_name = os.path.join(out_dir, raw_name + ".entropy.neg.bw")

    out_cmd = "python ~/gscripts/gscripts/general/make_bigwig_entropy_files.py --ip_bam {} --input_bam {} --genome /projects/ps-yeolab/genomes/hg19/hg19.chrom.sizes --bw_pos {} --bw_neg {}".format(clip_fn, input_fn, pos_name, neg_name)
    return out_cmd

In [102]:
results = []
for name, row in merged_data.iterrows():
    clip_fn = row.CLIP
    raw_name = os.path.splitext(os.path.basename(clip_fn))[0]
    pos_name = os.path.join(out_dir, raw_name + ".entropy.pos.bw")
    neg_name = os.path.join(out_dir, raw_name + ".entropy.neg.bw")

    if not (os.path.exists(pos_name) and os.path.exists(neg_name)):
        results.append(run_entropy_bw(row.CLIP, row.INPUT))

In [103]:
len(results)

167

In [93]:
merged_data = merged_data.sort_index()

In [101]:
for name, row in  merged_data.loc[['596', '460', '649', '654', '862']].iterrows():   
    clip_fn = row.CLIP
    raw_name = os.path.splitext(os.path.basename(clip_fn))[0]
    pos_name = os.path.join(out_dir, raw_name + ".entropy.pos.bw")
    neg_name = os.path.join(out_dir, raw_name + ".entropy.neg.bw")

    if not (os.path.exists(pos_name) and os.path.exists(neg_name)):
        print run_entropy_bw(row.CLIP, row.INPUT)

In [104]:
num_slices = 500
full_process_list = []
for x in range(num_slices):
    cur_list = results[x::num_slices]
    if len(cur_list) > 0:
        full_process_list.append(" && ".join(cur_list))

In [105]:
job_name = "entropy_input_norm"
job = qtools.Submitter(commands=full_process_list, 
                 job_name="{}".format(job_name), 
                 sh_filename="/home/gpratt/projects/encode/scripts/{}.sh".format(job_name),
                array=True,
                walltime="5:00:00",
                out_filename="/home/gpratt/projects/encode/scripts/{}.out".format(job_name),
                err_filename="/home/gpratt/projects/encode/scripts/{}.err".format(job_name),
                queue="condo",
                      ppn=16)
job.job()

print "/home/gpratt/projects/encode/scripts/{}.sh".format(job_name)

/home/gpratt/projects/encode/scripts/entropy_input_norm.sh


running 167 tasks as an array-job.


In [26]:
from gscripts.general import make_bigwig_entropy_files

In [31]:
%load_ext autoreload
%autoreload 2

In [56]:
reload(make_bigwig_entropy_files)

<module 'gscripts.general.make_bigwig_entropy_files' from '/home/gpratt/gscripts/gscripts/general/make_bigwig_entropy_files.pyc'>

In [49]:

def get_stranded_coverage(bedtool):

    pos_df = bedtool.genome_coverage(dz=True, split=True, strand="+").to_dataframe(names=['chrom', 'loc', 'coverage'])
    neg_df = bedtool.genome_coverage(dz=True, split=True, strand="-").to_dataframe(names=['chrom', 'loc', 'coverage'])

    return pos_df, neg_df

In [53]:
bam_fn = "/home/gpratt/ad-hoc/chrM_test.bam"
input_bam_fn = "/home/gpratt/ad-hoc/chrM_input_test.bam"

In [70]:
ip_bam = pybedtools.BedTool(bam_fn)
input_bam = pybedtools.BedTool(input_bam_fn)

pos_ip_df, neg_ip_df = make_bigwig_entropy_files.get_stranded_coverage(ip_bam)
pos_input_df, neg_input_df = make_bigwig_entropy_files.get_stranded_coverage(input_bam)

total_ip = pos_ip_df.coverage.sum() + neg_ip_df.coverage.sum()
total_input = pos_input_df.coverage.sum() + neg_input_df.coverage.sum()


In [71]:
import gc

In [72]:
pos_df = make_bigwig_entropy_files.combine_ip_input_coverage(pos_ip_df, pos_input_df)

del pos_ip_df, pos_input_df
gc.collect()


489

In [73]:
pos_df = make_bigwig_entropy_files.calculate_coverage_probability(pos_df, total_ip, total_input)
pos_df = make_bigwig_entropy_files.calculate_entropy(pos_df)
pos_bg = make_bigwig_entropy_files.write_bg(pos_df)
make_bigwig_entropy_files.write_bw(pos_bg, "/home/gpratt/ad-hoc/foo.pos.bw", '/projects/ps-yeolab/genomes/hg19/hg19.chrom.sizes')


bedGraphToBigWig /state/partition1/gpratt/8681503.tscc-mgr.local/tmp3rf8Ad /projects/ps-yeolab/genomes/hg19/hg19.chrom.sizes /home/gpratt/ad-hoc/foo.pos.bw
